# Install packages

In [ ]:
from google.colab import drive # import drive from google colab
drive.mount("/content/drive") 
!pip install neo
!pip install matplotlib_scalebar
!pip install ffmpeg
!pip install Ipython --upgrade
%load_ext autoreload
%autoreload 2

# Import packages

In [ ]:
# if using google drive
%cd /content/drive/MyDrive/ff_repo/Multifirefly-Project

In [ ]:
%cd /Users/dusiyi/Documents/Multifirefly-Project

In [2]:
%load_ext autoreload
%autoreload 2

import sys
if not '/Users/dusiyi/Documents/Multifirefly-Project/multiff_analysis/methods' in sys.path:
    sys.path.append('/Users/dusiyi/Documents/Multifirefly-Project/multiff_analysis/methods')
from data_wrangling import basic_func, process_raw_data, base_processing_class, monkey_data_classes, combine_info_utils
from pattern_discovery import pattern_by_trials, pattern_by_points, make_ff_dataframe, ff_dataframe_utils, pattern_by_trials, pattern_by_points, cluster_analysis, organize_patterns_and_features, category_class
from decision_making_analysis.decision_making import decision_making_class, decision_making_utils, plot_decision_making, intended_targets_classes
from decision_making_analysis.GUAT import GUAT_helper_class, GUAT_collect_info_class, GUAT_combine_info_class, process_GUAT_trials_class, GUAT_and_TAFT
from decision_making_analysis import free_selection, replacement, trajectory_info
from null_behaviors import show_null_trajectory, find_best_arc, curvature_utils, curvature_class, curv_of_traj_utils
from machine_learning import machine_learning_utils, hyperparameter_tuning_class
from machine_learning.RL.env_related import env_for_lstm, env_utils, base_env, collect_agent_data_utils
from machine_learning.RL.lstm import GRU_functions, LSTM_functions
from machine_learning.RL.SB3 import interpret_neural_network, sb3_for_multiff_class, rl_for_multiff_utils, SB3_functions
from visualization import plot_trials, animation_func, plot_behaviors_utils, monkey_heading_functions
from planning_analysis.show_planning import alt_ff_utils, show_planning_class, show_planning_utils
from planning_analysis.show_planning.get_stops_near_ff import find_stops_near_ff_class, find_stops_near_ff_utils, plot_stops_near_ff_class, plot_stops_near_ff_utils, plot_monkey_heading_helper_class, stops_near_ff_based_on_ref_class
from planning_analysis.test_params_for_planning import params_test_combos_class, params_utils
from visualization.plotly_tools import plotly_for_monkey, plotly_for_scatterplot, plotly_preparation, plotly_for_correlation
from visualization.dash_tools import dash_prep_class, dash_utils, dash_utils, dash_comparison_class, dash_params_class
from visualization.dash_tools.dash_main_class_methods import dash_main_class
from importlib import reload
from non_behavioral_analysis import eye_positions
import os
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm
from os.path import exists
import seaborn as sns
import math
import copy
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import tqdm
import itertools
import plotly.express as px
from scipy.stats import rankdata
from scipy import stats
from IPython.display import HTML
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.datasets import make_multilabel_classification
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, hamming_loss, multilabel_confusion_matrix, fbeta_score, precision_score, recall_score
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, VotingClassifier, BaggingClassifier, GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from math import pi
from scipy.ndimage import gaussian_filter1d
from statsmodels.graphics.tsaplots import plot_acf
import warnings
import os, sys
from sklearn.exceptions import ConvergenceWarning
import plotly.graph_objects as go

plt.rcParams["animation.html"] = "html5"
os.environ['KMP_DUPLICATE_LIB_OK']='True'
rc('animation', html='jshtml')
matplotlib.rcParams.update(matplotlib.rcParamsDefault)
matplotlib.rcParams['animation.embed_limit'] = 2**128
pd.set_option('display.float_format', lambda x: '%.5f' % x)
np.set_printoptions(suppress=True)
pd.options.display.max_rows = 101

# Basic data

## Retrieve monkey data

In [ ]:
PLAYER = "monkey"
monkey_data_folder_path = "all_monkey_data/raw_monkey_data/individual_monkey_data/monkey_Bruno/data_0322"
data_item = monkey_data_classes.ProcessMonkeyData(raw_data_folder_path=monkey_data_folder_path)
data_item.retrieve_or_make_monkey_data(exists_ok=True)
data_item.make_or_retrieve_ff_dataframe(num_missed_index=0, exists_ok=True)

data_item.find_patterns()
data_item.make_PlotTrials_args()

monkey_information = data_item.monkey_information
ff_dataframe = data_item.ff_dataframe

ff_life_sorted = data_item.ff_life_sorted
ff_real_position_sorted = data_item.ff_real_position_sorted
ff_believed_position_sorted = data_item.ff_believed_position_sorted
cluster_around_target_indices = data_item.cluster_around_target_indices
ff_caught_T_sorted = data_item.ff_caught_T_sorted
caught_ff_num = len(ff_caught_T_sorted)
ff_flash_sorted = data_item.ff_flash_sorted
ff_flash_end_sorted = data_item.ff_flash_end_sorted
max_point_index = data_item.max_point_index
min_point_index = data_item.min_point_index

monkey_name = data_item.model_folder_name.split('/')[2]
data_name = data_item.model_folder_name.split('/')[3]


PlotTrials_args = (monkey_information, ff_dataframe, ff_life_sorted, ff_real_position_sorted, ff_believed_position_sorted, cluster_around_target_indices, ff_caught_T_sorted)


plot_polar_args = (monkey_information,
                    ff_dataframe, 
                    ff_life_sorted,
                    ff_real_position_sorted,
                    ff_caught_T_sorted,
                    ff_flash_sorted,)


trial_total_num = 2
PLAYER = "monkey"

classic_plot_kwargs = {'player': PLAYER,
                       'show_stops': True,
                       'show_believed_target_positions': True,
                       'show_reward_boundary': True,
                       'show_scale_bar': True,
                       'show_eye_positions': True,
                       'show_eye_positions_on_the_right': True,
                       'show_connect_path_eye_positions': True,
                       #=== below is different from animation_plot_kwargs
                       'hitting_arena_edge_ok': False,
                       'trial_too_short_ok': False}

combined_plot_kwargs = {'player': PLAYER,
                        'combined_plot': True,
                        'show_alive_fireflies': False,
                        'show_title': False,
                        'show_start': False}

plot_chunks_kwargs = {'player': PLAYER,
                      'show_stops': True,
                      'show_believed_target_positions': True,
                      'show_reward_boundary': True,
                      'show_scale_bar': True,
                      'hitting_arena_edge_ok': True,
                      'trial_too_short_ok': True,
                      'show_connect_path_ff': True,
                      'trail_color_var': 'abs_ddw',
                      'show_colorbar': True}


plot_polar_kwargs = {'rmax': 400,
                    'show_visible_ff': True,
                    'hitting_arena_edge_ok': True,
                    'return_axes': True,
                    'colors_show_overall_time': True,
                    'show_ff_in_memory': True,
                    'show_target_at_being_caught': True
}

print("player is", PLAYER)

animation_plot_kwargs = classic_plot_kwargs.copy()
animation_plot_kwargs['as_part_of_animation'] = True
animation_plot_kwargs['show_eye_positions_on_the_right'] = False   
animation_plot_kwargs['hitting_arena_edge_ok'] = True
animation_plot_kwargs['trial_too_short_ok'] = True
animation_plot_kwargs['images_dir'] = None


  

data_item.make_PlotTrials_kargs(classic_plot_kwargs, combined_plot_kwargs, animation_plot_kwargs)


plotting_kwargs = {'player': 'monkey',
 'show_stops': True,
 'show_believed_target_positions': True,
 'show_reward_boundary': True,
 'show_scale_bar': True,
 'hitting_arena_edge_ok': True,
 'trial_too_short_ok': True,
 'show_connect_path_ff': True,
 'vary_color_for_connecting_path_ff': True,
 'show_points_when_ff_stop_being_visible': False,
 #'show_connect_path_ff_memory': True,
 'show_alive_fireflies': False,
 'show_visible_fireflies': True,
 'show_in_memory_fireflies': True,
 'connect_path_ff_max_distance': 500,
 #'show_connect_path_ff_except_targets': True,
 'adjust_xy_limits': True,
 'show_null_agent_trajectory': True,
 'show_only_ff_that_monkey_has_passed_by_closely': True,
 'show_null_trajectory_reaching_boundary_ok': False,
 'vary_color_for_connecting_path_ff': False}

In [ ]:
# PlotTrials_args = (monkey_information, ff_dataframe, ff_life_sorted, ff_real_position_sorted, ff_believed_position_sorted, cluster_around_target_indices, ff_caught_T_sorted)


# plot_polar_args = (monkey_information,
#                     ff_dataframe, 
#                     ff_life_sorted,
#                     ff_real_position_sorted,
#                     ff_caught_T_sorted,
#                     ff_flash_sorted,)

# data_item.make_PlotTrials_args()
# data_item.make_PlotTrials_kargs(classic_plot_kwargs, combined_plot_kwargs, animation_plot_kwargs)


## ff_basic_info

In [ ]:
# make ff_basic_info
num_missing_elements = len(ff_real_position_sorted) - len(ff_caught_T_sorted)
ff_caught_T_sorted_extended = np.concatenate([ff_caught_T_sorted, np.repeat(np.nan, num_missing_elements)])
ff_believed_position_sorted_extended = np.concatenate([ff_believed_position_sorted, np.repeat(np.nan, num_missing_elements*2).reshape(-1,2)])

ff_basic_info_bundle = (ff_real_position_sorted, ff_believed_position_sorted_extended, ff_life_sorted, ff_caught_T_sorted_extended, ff_flash_end_sorted)

ff_basic_info = pd.DataFrame({'ff_index': range(len(ff_real_position_sorted)),
                            'caught_T': ff_caught_T_sorted_extended,
                            'real_x': ff_real_position_sorted[:,0],
                            'real_y': ff_real_position_sorted[:,1],
                            'believed_x': ff_believed_position_sorted_extended[:,0],
                            'believed_y': ff_believed_position_sorted_extended[:,1],
                            'life_start': ff_life_sorted[:,0],
                            'life_end': ff_life_sorted[:,1],
                            'flash_end': ff_flash_end_sorted,

})


ff_basic_info.head(3)

## plotting kwargs

In [ ]:
null_arcs_plotting_kwargs = {'player': 'monkey',
    'show_stops': True,
    'show_believed_target_positions': True,
    'show_reward_boundary': True,
    'show_scale_bar': True,
    'hitting_arena_edge_ok': True,
    'trial_too_short_ok': True,
    'show_connect_path_ff': False,
    'vary_color_for_connecting_path_ff': True,
    'show_points_when_ff_stop_being_visible': False,
    'show_alive_fireflies': False,
    'show_visible_fireflies': True,
    'show_in_memory_fireflies': True,
    'connect_path_ff_max_distance': 500,
    'adjust_xy_limits': True,
    'show_null_agent_trajectory': False,
    'show_only_ff_that_monkey_has_passed_by_closely': False,
    'show_null_trajectory_reaching_boundary_ok': False,
    'zoom_in': False,
    'truncate_part_before_crossing_arena_edge': True}

In [ ]:
stop!

# SNF (stops near ff) class

In [ ]:
import cProfile
cProfile.run('snf.streamline_organizing_info()', sort='cumtime')

## run class

In [5]:
raw_data_folder_path = "all_monkey_data/raw_monkey_data/individual_monkey_data/monkey_Bruno/data_0322"

In [ ]:

normalize = False
eliminate_outliers = False
use_curvature_to_ff_center = False
curv_of_traj_mode = 'distance'
window_for_curv_of_traj=[-25, 25]

#data_item_info = find_stops_near_ff_utils.extract_key_info_from_data_item_for_stops_near_ff_class(data_item)
snf = stops_near_ff_based_on_ref_class.StopsNearFFBasedOnRef(raw_data_folder_path=raw_data_folder_path)
snf.get_more_monkey_data()
snf.traj_curv_descr = 'Traj Curv: From Current Point to Right Before Stop'

snf.streamline_organizing_info(ref_point_mode='distance', ref_point_value=-120, 
                              #ref_point_mode='time after stop ff visible', ref_point_value=0.1, 
                               curv_of_traj_mode=curv_of_traj_mode, window_for_curv_of_traj=window_for_curv_of_traj, truncate_curv_of_traj_by_time_of_capture=True,
                               use_curvature_to_ff_center=use_curvature_to_ff_center,  eliminate_outliers=eliminate_outliers,
                               stops_near_ff_df_exists_ok=True)
snf.ax_for_corr = snf.find_relationships_from_info(normalize=normalize, show_plot=True)
snf.prepare_to_make_plots()

## Other plots

### see a distribution of how long before the stop did both become visible

In [ ]:
earliest_time_point_both_visible = snf.monkey_information.loc[snf.stops_near_ff_df['earlest_point_index_when_alt_ff_and_stop_ff_have_both_been_seen_bbas'], 'monkey_t'].values
duration_till_stop = snf.stops_near_ff_df['stop_time'].values - earliest_time_point_both_visible
sns.histplot(duration_till_stop, bins=100)
plt.show()


### percentage

In [12]:
snf._make_info_based_on_monkey_angle()

In [ ]:
(snf.info_based_on_monkey_angle_before_stop['dir_from_stop_ff_to_stop']==snf.info_based_on_monkey_angle_before_stop['dir_from_stop_ff_to_alt_ff']).sum()/len(snf.stops_near_ff_df)

In [ ]:
(snf.info_based_on_monkey_angle_when_alt_ff_last_seen['dir_from_stop_ff_to_stop']==snf.info_based_on_monkey_angle_when_alt_ff_last_seen['dir_from_stop_ff_to_alt_ff']).sum()/len(snf.stops_near_ff_df)

# hyperparameter tuning

The current calculation of curv_r/heading_r needs to be updated.

Also, the operation is currently only based on one monkey session.

In [16]:
# Potential hyperparameters:

# below can be changed to:
ref_point_mode = 'time' # or 'distance'
ref_point_value = -100 

#window_for_curv_of_traj=[-25, 25]
curv_of_traj_mode = 'distance'
curv_of_traj_lower_end = -25
curv_of_traj_upper_end = 25
truncate_curv_of_traj_by_time_of_capture=False,
eliminate_outliers=True
use_curvature_to_ff_center=False

## get tested_combo_df

In [ ]:
ptc.all_combo_short

In [ ]:
ptc._deal_with_rows_with_big_ff_angles(remove_i_o_modify_rows_with_big_ff_angles=True)

In [ ]:
ptc.find_relative_curvature()

In [ ]:
raw_data_folder_path = "all_monkey_data/raw_monkey_data/individual_monkey_data/monkey_Bruno/data_0322"
ptc = params_test_combos_class.ParamsTestCombos(raw_data_folder_path=raw_data_folder_path)
ptc.generate_all_combo_df() 
# save ptc.tested_combo_df into a csv file
tested_combo_df = ptc.test_all_set_of_hyperparameters(sample_size=1000)
tested_combo_df.to_csv(os.path.join(raw_data_folder_path, 'planning_hyperparameters_df.csv'))

In [37]:
raw_data_folder_path = 'all_monkey_data/raw_monkey_data/individual_monkey_data/monkey_Bruno/data_0312'

In [38]:
tested_combo_df = pd.read_csv(os.path.join(raw_data_folder_path, 'planning_hyperparameters_df.csv'))

## inspection

In [39]:
tested_combo_df['ref_point_mode_short'] = tested_combo_df['ref_point_mode'].apply(lambda x: x.split(' ')[0])
tested_combo_df['curv_of_traj_mode_short'] = tested_combo_df['curv_of_traj_mode'].apply(lambda x: x.split(' ')[0])
tested_combo_df['same_mode'] = tested_combo_df['ref_point_mode_short'] == tested_combo_df['curv_of_traj_mode_short']

In [ ]:
# # take out the subset where the r value is above 2 std of shuffled r values and see what's going on in each
tested_combo_df['shuffled_curv_r_mean_plus_2_std'] = tested_combo_df['shuffled_curv_r_mean'] + tested_combo_df['shuffled_curv_r_std']*2
sub_df = tested_combo_df[tested_combo_df['curv_r'] > tested_combo_df['shuffled_curv_r_mean_plus_2_std']]
sub_df

In [ ]:
# take out the subset where the r value is above 2 std of shuffled r value and see what's going on in each
tested_combo_df['shuffled_heading_r_mean_plus_2_std'] = tested_combo_df['shuffled_heading_r_mean'] + tested_combo_df['shuffled_heading_r_std']*2
sub_df = tested_combo_df[tested_combo_df['heading_r'] > tested_combo_df['shuffled_heading_r_mean_plus_2_std']].copy()
sub_df

# Dash

## main

In [ ]:

ref_point_params = {'ref_point_mode': 'distance',
                    'ref_point_value': -150}


curv_of_traj_params = {'curv_of_traj_mode': 'distance',
                        'window_for_curv_of_traj': [-25, 25],
                        'truncate_curv_of_traj_by_time_of_capture': False}

overall_params = {'eliminate_outliers': True,
                  'use_curvature_to_ff_center': False,
                  'normalize': False,
                  'remove_i_o_modify_rows_with_big_ff_angles': True,
                  'change_units_to_degrees_per_m': True,
                  'heading_instead_of_curv': True}  

monkey_plot_params = {
    "show_visible_fireflies": True,
    "show_in_memory_fireflies": False,
    "show_monkey_heading": False,
    "show_visible_segments": True,
    "show_traj_portion": False,
    "show_null_arcs_to_ff": False,
    "show_stops": True,
    "show_all_eye_positions": True,
    "show_current_eye_positions": True,
    "show_eye_positions_for_both_eyes": True,
}

scatter_plot_params = {
    "use_two_y_axes": False,
    "show_alt_ff_curv_in_scatterplot": True,
    "show_stop_ff_curv_in_scatterplot": True,
}

                     
dc = dash_main_class.DashMainPlots(raw_data_folder_path=raw_data_folder_path)

dc.prepare_to_make_dash_for_main_plots(monkey_plot_params=monkey_plot_params,
                                        scatter_plot_params=scatter_plot_params,
                                        ref_point_params=ref_point_params,
                                        curv_of_traj_params=curv_of_traj_params,
                                        overall_params=overall_params)

#dc.fig.show()
#dc.fig_scatter_combd.show()


In [ ]:
dc.make_dash_for_main_plots(show_trajectory_scatter_plot=False)

## comparison

shuffle alt_ff

In [ ]:
dcom = dash_comparison_class.DashComparison(raw_data_folder_path=raw_data_folder_path)

            
ref_point_params = {'ref_point_mode': 'distance',
                    'ref_point_value': -150}


curv_of_traj_params = {'curv_of_traj_mode': 'distance',
                        'window_for_curv_of_traj': [-75, 75],
                        'truncate_curv_of_traj_by_time_of_capture': False}

overall_params = {'eliminate_outliers': True,
                  'use_curvature_to_ff_center': False,
                  'normalize': False,
                  'remove_i_o_modify_rows_with_big_ff_angles': True,
                  'change_units_to_degrees_per_m': True,
                  'heading_instead_of_curv': True}  

        
dcom.prepare_to_make_dash_for_comparison(ref_point_params,
                                         curv_of_traj_params,
                                         overall_params,)

#dcom.make_dash_for_comparison()



              

In [ ]:
dcom.make_dash_for_comparison()

## compare parameters

The visualization looks good, but the code certainly needs to be updated.

In [ ]:
try:
    tested_combo_df
except:
    raise ValueError("tested_combo_df not defined")

pad = dash_params_class.ParamsDash(raw_data_folder_path=raw_data_folder_path)
                                             
pad.ref_point_mode = 'time'
pad.curv_of_traj_mode = 'time'
tested_combo_df_sub = tested_combo_df[tested_combo_df['curv_r'] != 0].copy()
pad.prepare_for_dash(tested_combo_df_sub)
pad.make_dash_for_params()

# Experiment on curv

I wanted to see why the r value of the shuffled data is still so high...

I realized that traj and stop already have a very high correlation; in fact, traj - alt and stop - alt has a lower correlation but not by much. Therefore, no matter how alt gets shuffled, there's still gonna be correlation that remains. 

I guess this is the difficulty associated with looking for correlation between difference of vars.

### corr of diff between vars 

In [46]:
stop_ff_counted_df = dc.stop_ff_counted_df.copy()
alt_ff_counted_df = dc.alt_ff_counted_df.copy()
# also make a copy of alt_ff_counted_df and shuffle it
alt_ff_counted_df2 = dc.alt_ff_counted_df.copy()
alt_ff_counted_df2[['curv_to_ff_center', 'optimal_curvature']] = alt_ff_counted_df2[['curv_to_ff_center', 'optimal_curvature']].sample(frac=1).values

alt = alt_ff_counted_df['curv_to_ff_center'].values
alt2 = alt_ff_counted_df2['curv_to_ff_center'].values
traj = alt_ff_counted_df['curvature_of_traj'].values
stop = stop_ff_counted_df['curv_to_ff_center'].values

In [47]:
def show_relationships(x, y):
    sns.scatterplot(x=x, y=y)
    slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)
    print('slope:', slope)
    print('r_value:', r_value)
    return 

In [ ]:
x = traj-alt
y = stop-alt
show_relationships(x, y)

In [ ]:
x = traj
y = stop-alt
show_relationships(x, y)

In [ ]:
x = traj-alt2
y = stop-alt
show_relationships(x, y)

In [ ]:
x = traj
y = stop
show_relationships(x, y)

In [ ]:
x = stop
y = alt
show_relationships(x, y)

### corr of vars

In [ ]:
dc.fig_scatter_combd

In [ ]:
sns.histplot(stop - traj)
sns.histplot(alt - traj)

In [ ]:
sns.scatterplot(x=traj, y=stop)
slope, intercept, r_value, p_value, std_err = stats.linregress(traj, stop)
print('slope:', slope)
print('r_value:', r_value)

In [ ]:
sns.scatterplot(x=traj, y=alt)
slope, intercept, r_value, p_value, std_err = stats.linregress(traj, alt)
print('slope:', slope)
print('r_value:', r_value)


In [ ]:
sns.scatterplot(x=traj, y=alt2)
slope, intercept, r_value, p_value, std_err = stats.linregress(traj, alt2)
print('slope:', slope)
print('r_value:', r_value)

In [ ]:
sns.scatterplot(x=traj, y=alt)
sns.scatterplot(x=stop, y=alt)
plt.show()

# Other plots

## compare 3 heading

In [ ]:
dcom._find_relative_heading_info()
sns.histplot(dcom.d_heading_alt, bins=20, color='r', alpha=0.5, stat='density', label='alt', kde=True)
sns.histplot(dcom.d_heading_stop, bins=20, color='g', alpha=0.5, stat='density', label='stop', kde=True)
sns.histplot(dcom.d_heading_of_traj, bins=20, color='b', alpha=0.5, stat='density', label='traj', kde=True)
plt.legend()
plt.show()


In [ ]:
import plotly.graph_objects as go

dc._find_relative_heading_info()
x_array = np.arange(len(dc.d_heading_alt))

# make a plotly figure for each of the three y_array above, with x_array being the same:
fig = go.Figure()
fig.add_trace(go.Scatter(x=x_array, y=dcom.d_heading_alt, mode='lines', name='alt'))
fig.add_trace(go.Scatter(x=x_array, y=dcom.d_heading_stop, mode='lines', name='stop'))
fig.add_trace(go.Scatter(x=x_array, y=dcom.d_heading_of_traj, mode='lines', name='traj'))

fig.show()

## what's wrong with monkey angle?

In [ ]:
duration_to_plot = [27, 31.944755]
trajectory_df, R = plotly_preparation.make_trajectory_df(dc.PlotTrials_args, duration_to_plot=duration_to_plot)
show_stop_point_indices = trajectory_df.loc[trajectory_df['monkey_speeddummy']==0, 'point_index'].values
mheading = monkey_heading_functions.find_mheading_in_xy(trajectory_df[['point_index']].values, dc.monkey_information)
triangle_df = monkey_heading_functions.turn_mheading_into_triangle_df(mheading, R, point_index=None)


fig = go.Figure()
#fig = plot_trajectory_data(fig, trajectory_df, hoverdata_multi_columns=[])
fig = plotly_for_monkey.plot_trajectory_data(fig, trajectory_df, hoverdata_multi_columns=[])
fig = plotly_for_monkey.plot_stops_in_plotly(fig, trajectory_df, show_stop_point_indices, hoverdata_multi_columns=[])
fig = monkey_heading_functions.plot_triangles_to_show_monkey_heading_in_xy_in_plotly(fig, triangle_df, trace_name_prefix=None, 
                                                                                      color='red', linewidth=0.5)
# make x and y axis have the same scale
fig.update_yaxes(scaleanchor="x", scaleratio=1) 
fig

## same mode or different: which is better

In [87]:
tested_combo_df_p = params_utils.process_tested_combo_df(tested_combo_df)

In [ ]:
tested_combo_df_p.columns

In [ ]:
tested_combo_df.sort_values(by='curv_r_z_score', ascending=False)[['same_mode', 'curv_r_z_score', 'use_curvature_to_ff_center']].head(20)

In [ ]:
tested_combo_df.sort_values(by='heading_r_z_score', ascending=False)[['same_mode', 'heading_r_z_score', 'use_curvature_to_ff_center']].head(20)